In [1]:
from __future__ import print_function, division
import os
import torchvision
import torch
from skimage import io, transform
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
from math import log10, pi
import time

import utils
from datasetsMultiple import DatasetMultiple
from vgg import Vgg16

c:\users\adams\appdata\local\programs\python\python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\adams\appdata\local\programs\python\python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\users\adams\appdata\local\programs\python\python39\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
class MFFNet(torch.nn.Module):
    def __init__(self):
        super(MFFNet, self).__init__()
        
        self.conv1 = ConvLayer(3, 32, kernel_size=9, stride=1)
        self.in1 = torch.nn.InstanceNorm2d(32, affine=True)
        self.conv2 = ConvLayer(32, 64, kernel_size=3, stride=2)
        self.in2 = torch.nn.InstanceNorm2d(64, affine=True)
        self.conv3 = ConvLayer(64, 128, kernel_size=3, stride=2)
        self.in3 = torch.nn.InstanceNorm2d(128, affine=True)
        
        # Residual layers
        self.res1 = ResidualBlock(128)
        self.res2 = ResidualBlock(128)
        self.res3 = ResidualBlock(128)
        self.res4 = ResidualBlock(128)
        self.res5 = ResidualBlock(128)
        self.res6 = ResidualBlock(128)
        self.res7 = ResidualBlock(128)
        self.res8 = ResidualBlock(128)
        self.res9 = ResidualBlock(128)
        self.res10 = ResidualBlock(128)
        self.res11 = ResidualBlock(128)
        self.res12 = ResidualBlock(128)
        self.res13 = ResidualBlock(128)
        self.res14 = ResidualBlock(128)
        self.res15 = ResidualBlock(128)
        self.res16 = ResidualBlock(128)
        
        self.deconv1 = UpsampleConvLayer(128*2, 64, kernel_size=3, stride=1, upsample=2)
        self.in4 = torch.nn.InstanceNorm2d(64, affine=True)
        self.deconv2 = UpsampleConvLayer(64*2, 32, kernel_size=3, stride=1, upsample=2)
        self.in5 = torch.nn.InstanceNorm2d(32, affine=True)
        self.deconv3 = ConvLayer(32*2, 3, kernel_size=9, stride=1)

        self.relu = torch.nn.ReLU()
        
        # Attentions
        self.att1 = AttentionBlock(128, 128, 64)
        self.att2 = AttentionBlock(64, 64, 32)
        self.att3 = AttentionBlock(32, 32, 16)

    
    def forward(self, X):
        o1 = self.relu(self.conv1(X))
        o2 = self.relu(self.conv2(o1))
        o3 = self.relu(self.conv3(o2))

        y = self.res1(o3)
        y = self.res2(y)
        y = self.res3(y)
        y = self.res4(y)
        y = self.res5(y)
        y = self.res6(y)
        y = self.res7(y)
        y = self.res8(y)
        y = self.res9(y)
        y = self.res10(y)
        y = self.res11(y)
        y = self.res12(y)
        y = self.res13(y)
        y = self.res14(y)
        y = self.res15(y)
        y = self.res16(y)
        
        o3 = self.att1(y, o3)
        in1 = torch.cat((y, o3), 1)
        y = self.relu(self.deconv1(in1))

        o2 = self.att2(y, o2)
        in2 = torch.cat((y, o2), 1)
        y = self.relu(self.deconv2(in2))

        o1 = self.att3(y, o1)
        in3 = torch.cat((y, o1), 1)
        y = self.deconv3(in3)
        
        return y



class ConvLayer(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super(ConvLayer, self).__init__()
        reflection_padding = kernel_size // 2
        self.reflection_pad = torch.nn.ReflectionPad2d(reflection_padding)
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride)
    
    def forward(self, x):
        out = self.reflection_pad(x)
        out = self.conv2d(out)
        return out



class ResidualBlock(torch.nn.Module):
    
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in1 = torch.nn.InstanceNorm2d(channels, affine=True)
        self.conv2 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in2 = torch.nn.InstanceNorm2d(channels, affine=True)
        self.relu = torch.nn.ReLU()
    
    def forward(self, x):
        residual = x
        out = self.relu(self.conv1(x))
        out = self.conv2(out)
        out = out + residual
        return out



class UpsampleConvLayer(torch.nn.Module):
    
    def __init__(self, in_channels, out_channels, kernel_size, stride, upsample=None):
        super(UpsampleConvLayer, self).__init__()
        self.upsample = upsample
        reflection_padding = kernel_size // 2
        self.reflection_pad = torch.nn.ReflectionPad2d(reflection_padding)
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride)
    
    def forward(self, x):
        x_in = x
        if self.upsample:
            x_in = torch.nn.functional.interpolate(x_in, mode='nearest', scale_factor=self.upsample)
        out = self.reflection_pad(x_in)
        out = self.conv2d(out)
        return out
    


class AttentionBlock(torch.nn.Module):
    
    def __init__(self, Fg, Fl, Fint):
        super(AttentionBlock, self).__init__()               
        self.Wg = nn.Sequential(nn.Conv2d(Fg, Fint, kernel_size=1, stride=1, padding=0), nn.BatchNorm2d(Fint))
        self.Wx = nn.Sequential(nn.Conv2d(Fl, Fint, kernel_size=1, stride=1, padding=0), nn.BatchNorm2d(Fint))
        self.relu = nn.ReLU()
        self.psi = nn.Sequential(nn.Conv2d(Fint, 1, kernel_size=1, stride=1, padding=0), nn.BatchNorm2d(1), nn.Sigmoid())
        
    def forward(self, g, x):
        g1 = self.Wg(g)   
        x1 = self.Wx(x)
        y = self.relu(g1 + x1)  
        y = self.psi(y)
        return torch.mul(y, x)

In [3]:
train_dataset = DatasetMultiple('rgb train 100/', 'rgb train 100/', '380 train a 100/', '640 train a 100/', True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=16,
                                           shuffle=True,
                                           num_workers=4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
imageFilter = MFFNet().to(device).float()

# Initializing VGG16 model for perceptual loss
VGG = Vgg16(requires_grad=False)
VGG = VGG.to(device)


num_epochs = 1200
learning_rate = 1e-4

criterion_img = nn.MSELoss()
criterion_vgg = nn.MSELoss()

optimizer = torch.optim.Adam(imageFilter.parameters(), lr=learning_rate)
total_step = len(train_loader)


start_time = time.time()
previous_time = start_time
for epoch in range(num_epochs):
    loss_tol = 0
    loss_tol_vgg  = 0
    loss_tol_l2   = 0
    
    if epoch == 300:
        learning_rate = 1e-4
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
        
    if epoch == 600:
        learning_rate = 1e-5 #previously 1e-6, with 300 as 1e-5
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
    
    for i, im in enumerate(train_loader):
        inputs = im[0].float().to(device)
        target = im[1].float().to(device)
        
        outputs = imageFilter(inputs)
        
        loss_l2 = criterion_img( outputs, target )
        
        outputs_n = utils.normalize_ImageNet_stats(outputs)
        target_n  = utils.normalize_ImageNet_stats(target)
        
        feature_o = VGG(outputs_n, 3)
        feature_t = VGG(target_n, 3)
        VGG_loss = []
        for l in range(3+1):
            VGG_loss.append( criterion_vgg(feature_o[l], feature_t[l]) )
        
        loss_vgg = sum(VGG_loss)
        loss = loss_l2 + 0.01*loss_vgg
    
        loss_tol += loss.item()
        
        loss_tol_vgg  += loss_vgg
        loss_tol_l2   += loss_l2
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print ( 'Epoch [{}/{}], Training Loss: {:.4f}, vgg Loss: {:.4f}, L2 Loss: {:.4f}, time: {:.4f}' .format(epoch+1, num_epochs, loss_tol, loss_tol_vgg, loss_tol_l2, time.time() - previous_time) )
    previous_time = time.time()
    
print("--- %0.4f seconds ---" % (time.time() - start_time)) 
torch.save(imageFilter.state_dict(), 'MFF-net_all3_a_new_100_1200epochs.ckpt')

Epoch [1/1200], Training Loss: 307.7778, vgg Loss: 23608.3438, L2 Loss: 71.6943, time: 7.4991
Epoch [2/1200], Training Loss: 86.6120, vgg Loss: 7045.3936, L2 Loss: 16.1580, time: 6.1299
Epoch [3/1200], Training Loss: 43.7618, vgg Loss: 3827.6487, L2 Loss: 5.4853, time: 6.1572
Epoch [4/1200], Training Loss: 24.1681, vgg Loss: 2045.1240, L2 Loss: 3.7168, time: 6.3490
Epoch [5/1200], Training Loss: 20.4471, vgg Loss: 1726.8345, L2 Loss: 3.1788, time: 6.8984
Epoch [6/1200], Training Loss: 13.7848, vgg Loss: 1174.2084, L2 Loss: 2.0428, time: 7.6637
Epoch [7/1200], Training Loss: 11.3603, vgg Loss: 975.6278, L2 Loss: 1.6040, time: 6.7759
Epoch [8/1200], Training Loss: 8.7645, vgg Loss: 755.9742, L2 Loss: 1.2047, time: 6.5011
Epoch [9/1200], Training Loss: 6.9145, vgg Loss: 604.1609, L2 Loss: 0.8729, time: 6.2949
Epoch [10/1200], Training Loss: 7.7816, vgg Loss: 680.0017, L2 Loss: 0.9816, time: 6.6113
Epoch [11/1200], Training Loss: 6.1232, vgg Loss: 531.7476, L2 Loss: 0.8058, time: 6.6988
Ep

Epoch [92/1200], Training Loss: 2.4300, vgg Loss: 205.9497, L2 Loss: 0.3705, time: 7.8561
Epoch [93/1200], Training Loss: 2.4145, vgg Loss: 205.5609, L2 Loss: 0.3589, time: 7.7754
Epoch [94/1200], Training Loss: 2.2502, vgg Loss: 190.7835, L2 Loss: 0.3424, time: 7.8321
Epoch [95/1200], Training Loss: 1.9611, vgg Loss: 164.9713, L2 Loss: 0.3114, time: 8.3739
Epoch [96/1200], Training Loss: 2.0868, vgg Loss: 171.9582, L2 Loss: 0.3672, time: 8.0797
Epoch [97/1200], Training Loss: 2.0947, vgg Loss: 175.8123, L2 Loss: 0.3366, time: 8.2531
Epoch [98/1200], Training Loss: 2.1551, vgg Loss: 185.5701, L2 Loss: 0.2994, time: 8.0496
Epoch [99/1200], Training Loss: 2.4991, vgg Loss: 201.0182, L2 Loss: 0.4889, time: 7.7776
Epoch [100/1200], Training Loss: 2.3438, vgg Loss: 198.0789, L2 Loss: 0.3630, time: 7.8165
Epoch [101/1200], Training Loss: 1.9846, vgg Loss: 166.3882, L2 Loss: 0.3207, time: 7.7216
Epoch [102/1200], Training Loss: 2.0223, vgg Loss: 171.0741, L2 Loss: 0.3116, time: 7.7301
Epoch [

Epoch [183/1200], Training Loss: 1.5759, vgg Loss: 131.7879, L2 Loss: 0.2581, time: 8.0510
Epoch [184/1200], Training Loss: 2.1162, vgg Loss: 171.9152, L2 Loss: 0.3970, time: 7.7402
Epoch [185/1200], Training Loss: 3.1633, vgg Loss: 236.9584, L2 Loss: 0.7937, time: 8.0591
Epoch [186/1200], Training Loss: 2.4741, vgg Loss: 196.0651, L2 Loss: 0.5134, time: 7.8739
Epoch [187/1200], Training Loss: 1.8193, vgg Loss: 153.7144, L2 Loss: 0.2821, time: 7.7645
Epoch [188/1200], Training Loss: 1.5868, vgg Loss: 138.6737, L2 Loss: 0.2001, time: 7.8483
Epoch [189/1200], Training Loss: 1.5700, vgg Loss: 132.6532, L2 Loss: 0.2435, time: 7.7781
Epoch [190/1200], Training Loss: 1.5951, vgg Loss: 134.2108, L2 Loss: 0.2530, time: 7.7836
Epoch [191/1200], Training Loss: 1.4756, vgg Loss: 128.8780, L2 Loss: 0.1868, time: 7.8752
Epoch [192/1200], Training Loss: 1.6616, vgg Loss: 141.3735, L2 Loss: 0.2479, time: 7.8172
Epoch [193/1200], Training Loss: 1.5239, vgg Loss: 124.9854, L2 Loss: 0.2740, time: 7.7928

Epoch [274/1200], Training Loss: 1.0448, vgg Loss: 94.4585, L2 Loss: 0.1002, time: 8.0232
Epoch [275/1200], Training Loss: 1.0461, vgg Loss: 94.6049, L2 Loss: 0.1000, time: 8.0470
Epoch [276/1200], Training Loss: 1.1792, vgg Loss: 105.5041, L2 Loss: 0.1241, time: 8.1004
Epoch [277/1200], Training Loss: 1.0449, vgg Loss: 93.3711, L2 Loss: 0.1112, time: 7.8586
Epoch [278/1200], Training Loss: 0.8887, vgg Loss: 80.2407, L2 Loss: 0.0863, time: 8.1460
Epoch [279/1200], Training Loss: 0.9788, vgg Loss: 87.8370, L2 Loss: 0.1004, time: 7.9041
Epoch [280/1200], Training Loss: 1.0342, vgg Loss: 93.5136, L2 Loss: 0.0990, time: 8.0524
Epoch [281/1200], Training Loss: 1.0637, vgg Loss: 96.2908, L2 Loss: 0.1008, time: 8.0873
Epoch [282/1200], Training Loss: 0.9364, vgg Loss: 85.4315, L2 Loss: 0.0821, time: 7.8765
Epoch [283/1200], Training Loss: 1.1473, vgg Loss: 103.2082, L2 Loss: 0.1152, time: 8.0931
Epoch [284/1200], Training Loss: 1.0709, vgg Loss: 98.2114, L2 Loss: 0.0888, time: 8.1169
Epoch [2

Epoch [365/1200], Training Loss: 0.8295, vgg Loss: 75.1429, L2 Loss: 0.0781, time: 8.4286
Epoch [366/1200], Training Loss: 0.8872, vgg Loss: 80.3351, L2 Loss: 0.0839, time: 8.3138
Epoch [367/1200], Training Loss: 1.2090, vgg Loss: 105.0205, L2 Loss: 0.1588, time: 8.8643
Epoch [368/1200], Training Loss: 0.9991, vgg Loss: 90.4870, L2 Loss: 0.0942, time: 8.6421
Epoch [369/1200], Training Loss: 1.0593, vgg Loss: 93.8334, L2 Loss: 0.1210, time: 10.4343
Epoch [370/1200], Training Loss: 1.1343, vgg Loss: 98.0773, L2 Loss: 0.1535, time: 13.1623
Epoch [371/1200], Training Loss: 1.3912, vgg Loss: 115.8058, L2 Loss: 0.2331, time: 13.7559
Epoch [372/1200], Training Loss: 1.3262, vgg Loss: 110.5332, L2 Loss: 0.2209, time: 13.4620
Epoch [373/1200], Training Loss: 1.2247, vgg Loss: 102.7617, L2 Loss: 0.1971, time: 13.2043
Epoch [374/1200], Training Loss: 1.1538, vgg Loss: 100.5476, L2 Loss: 0.1483, time: 13.2895
Epoch [375/1200], Training Loss: 1.1749, vgg Loss: 102.0527, L2 Loss: 0.1543, time: 13.43

...

Epoch [823/1200], Training Loss: 0.6421, vgg Loss: 59.3762, L2 Loss: 0.0484, time: 7.0539
Epoch [824/1200], Training Loss: 0.5476, vgg Loss: 51.0543, L2 Loss: 0.0370, time: 7.0613
Epoch [825/1200], Training Loss: 0.5582, vgg Loss: 51.8572, L2 Loss: 0.0397, time: 7.0258
Epoch [826/1200], Training Loss: 0.6472, vgg Loss: 59.6857, L2 Loss: 0.0504, time: 7.0068
Epoch [827/1200], Training Loss: 0.6215, vgg Loss: 57.5612, L2 Loss: 0.0459, time: 7.0218
Epoch [828/1200], Training Loss: 0.6681, vgg Loss: 61.8178, L2 Loss: 0.0499, time: 7.0180
Epoch [829/1200], Training Loss: 0.6901, vgg Loss: 63.4065, L2 Loss: 0.0560, time: 6.9735
Epoch [830/1200], Training Loss: 0.6155, vgg Loss: 57.1446, L2 Loss: 0.0441, time: 6.8011
Epoch [831/1200], Training Loss: 0.6162, vgg Loss: 56.8738, L2 Loss: 0.0474, time: 6.9789
Epoch [832/1200], Training Loss: 0.5533, vgg Loss: 51.5886, L2 Loss: 0.0374, time: 7.0876
Epoch [833/1200], Training Loss: 0.5765, vgg Loss: 53.6461, L2 Loss: 0.0400, time: 6.8923
Epoch [834

Epoch [915/1200], Training Loss: 0.6359, vgg Loss: 58.7833, L2 Loss: 0.0480, time: 6.9108
Epoch [916/1200], Training Loss: 0.5718, vgg Loss: 52.0592, L2 Loss: 0.0512, time: 6.9421
Epoch [917/1200], Training Loss: 0.5570, vgg Loss: 51.4845, L2 Loss: 0.0421, time: 6.8005
Epoch [918/1200], Training Loss: 0.5513, vgg Loss: 51.0200, L2 Loss: 0.0411, time: 7.1913
Epoch [919/1200], Training Loss: 0.6591, vgg Loss: 60.0144, L2 Loss: 0.0590, time: 6.9234
Epoch [920/1200], Training Loss: 0.5340, vgg Loss: 49.6309, L2 Loss: 0.0377, time: 7.0755
Epoch [921/1200], Training Loss: 0.5349, vgg Loss: 49.5203, L2 Loss: 0.0397, time: 6.8788
Epoch [922/1200], Training Loss: 0.5788, vgg Loss: 53.6068, L2 Loss: 0.0427, time: 6.9989
Epoch [923/1200], Training Loss: 0.5250, vgg Loss: 48.9229, L2 Loss: 0.0358, time: 7.1747
Epoch [924/1200], Training Loss: 0.4977, vgg Loss: 46.3457, L2 Loss: 0.0342, time: 7.1589
Epoch [925/1200], Training Loss: 0.5381, vgg Loss: 50.1064, L2 Loss: 0.0370, time: 7.6956
Epoch [926

Epoch [1006/1200], Training Loss: 0.5078, vgg Loss: 46.6642, L2 Loss: 0.0411, time: 7.4890
Epoch [1007/1200], Training Loss: 0.5510, vgg Loss: 51.1553, L2 Loss: 0.0395, time: 7.5036
Epoch [1008/1200], Training Loss: 0.5513, vgg Loss: 51.2121, L2 Loss: 0.0391, time: 7.4368
Epoch [1009/1200], Training Loss: 0.5332, vgg Loss: 49.4826, L2 Loss: 0.0384, time: 7.5017
Epoch [1010/1200], Training Loss: 0.5851, vgg Loss: 54.0737, L2 Loss: 0.0444, time: 7.2856
Epoch [1011/1200], Training Loss: 0.6987, vgg Loss: 63.9723, L2 Loss: 0.0590, time: 7.3847
Epoch [1012/1200], Training Loss: 0.5530, vgg Loss: 51.0202, L2 Loss: 0.0428, time: 7.4158
Epoch [1013/1200], Training Loss: 0.5385, vgg Loss: 49.5729, L2 Loss: 0.0428, time: 7.5995
Epoch [1014/1200], Training Loss: 0.5784, vgg Loss: 52.8345, L2 Loss: 0.0500, time: 7.4698
Epoch [1015/1200], Training Loss: 0.6134, vgg Loss: 56.7670, L2 Loss: 0.0457, time: 7.4518
Epoch [1016/1200], Training Loss: 0.5210, vgg Loss: 48.3365, L2 Loss: 0.0376, time: 7.4159

Epoch [1097/1200], Training Loss: 0.5557, vgg Loss: 51.3229, L2 Loss: 0.0425, time: 7.5006
Epoch [1098/1200], Training Loss: 0.5488, vgg Loss: 50.6637, L2 Loss: 0.0422, time: 7.2013
Epoch [1099/1200], Training Loss: 0.5508, vgg Loss: 50.7999, L2 Loss: 0.0428, time: 7.4399
Epoch [1100/1200], Training Loss: 0.5960, vgg Loss: 54.6335, L2 Loss: 0.0496, time: 7.4905
Epoch [1101/1200], Training Loss: 0.5402, vgg Loss: 50.1157, L2 Loss: 0.0391, time: 7.4259
Epoch [1102/1200], Training Loss: 0.5717, vgg Loss: 52.5230, L2 Loss: 0.0464, time: 7.2394
Epoch [1103/1200], Training Loss: 0.5889, vgg Loss: 54.4132, L2 Loss: 0.0447, time: 7.5125
Epoch [1104/1200], Training Loss: 0.5216, vgg Loss: 48.4415, L2 Loss: 0.0372, time: 7.6054
Epoch [1105/1200], Training Loss: 0.5301, vgg Loss: 48.8052, L2 Loss: 0.0421, time: 7.4479
Epoch [1106/1200], Training Loss: 0.5024, vgg Loss: 46.8423, L2 Loss: 0.0339, time: 7.4838
Epoch [1107/1200], Training Loss: 0.5322, vgg Loss: 49.2027, L2 Loss: 0.0401, time: 7.4981

Epoch [1188/1200], Training Loss: 0.4900, vgg Loss: 45.0304, L2 Loss: 0.0397, time: 7.5497
Epoch [1189/1200], Training Loss: 0.5064, vgg Loss: 46.9913, L2 Loss: 0.0365, time: 7.4805
Epoch [1190/1200], Training Loss: 0.4956, vgg Loss: 46.2934, L2 Loss: 0.0327, time: 7.4572
Epoch [1191/1200], Training Loss: 0.5047, vgg Loss: 46.7643, L2 Loss: 0.0370, time: 7.3196
Epoch [1192/1200], Training Loss: 0.5146, vgg Loss: 47.8016, L2 Loss: 0.0366, time: 7.5417
Epoch [1193/1200], Training Loss: 0.5400, vgg Loss: 50.2936, L2 Loss: 0.0371, time: 7.6745
Epoch [1194/1200], Training Loss: 0.6089, vgg Loss: 56.1797, L2 Loss: 0.0471, time: 7.4511
Epoch [1195/1200], Training Loss: 0.5786, vgg Loss: 53.5635, L2 Loss: 0.0430, time: 7.5288
Epoch [1196/1200], Training Loss: 0.5298, vgg Loss: 48.8570, L2 Loss: 0.0412, time: 7.4525
Epoch [1197/1200], Training Loss: 0.5222, vgg Loss: 48.7612, L2 Loss: 0.0346, time: 7.5422
Epoch [1198/1200], Training Loss: 0.5598, vgg Loss: 52.2411, L2 Loss: 0.0374, time: 7.5322